**Default fastai tta don't denorm prediction so works only for plain non spatial predictions:**

https://github.com/fastai/fastai/blob/d82d1f8fbd67e83f21eb128005646977dda70db6/fastai/vision/tta.py#L19



In [7]:
#default_exp segmentation.tta

In [8]:
%autosave 60 
import fastai; print(fastai.__version__)

Autosaving every 60 seconds
1.0.58.dev0


In [9]:
#export
from fastai.vision import *

In [10]:
from local.test import *

In [15]:
# export
from fastai.basic_train import _loss_func2activ
def _seg_tta_only(learn:Learner, ds_type:DatasetType=DatasetType.Valid) -> Iterator[List[Tensor]]:
    "Computes the outputs for non-flip and flip_lr augmented inputs"
    dl = learn.dl(ds_type)
    ds = dl.dataset
    old = ds.tfms
    try:
        pbar = master_bar(range(2))
        for i in pbar:
            tfm = [] # to remove random crop resize aug
            if i: tfm.append(flip_lr(p=1.))
            ds.tfms = tfm
            yield get_preds(learn.model, dl, pbar=pbar, activ=_loss_func2activ(learn.loss_func))[0]
    finally: ds.tfms = old

Learner.seg_tta_only = _seg_tta_only

# flip_lr TTA preds
def _seg_TTA(learn, ds_type=DatasetType.Valid):
    "Takes average of original and flip_lr"
    orig_preds, flip_lr_preds = list(learn.seg_tta_only(learn, ds_type))
    flip_lr_preds = torch.stack([torch.flip(o, dims=[-1]) for o in flip_lr_preds], dim=0)
    avg_preds = (orig_preds + flip_lr_preds)/2
    return avg_preds

Learner.segTTA = _seg_TTA

In [13]:
# export


### export

In [6]:
from local.notebook.export import notebook2script
notebook2script(all_fs=True)

Converted 00_test.ipynb.
Converted 01_script.ipynb.
Converted 02_scheduler.ipynb.
Converted 03_callbacks.ipynb.
Converted 04_optimizers_optimizers.ipynb.
Converted 10_segmentation_dataset.ipynb.
Converted 11_segmentation_losses_mulitlabel.ipynb.
Converted 11b_segmentation_losses_binary.ipynb.
Converted 12_segmentation_metrics.ipynb.
Converted 13_segmentation_models.ipynb.
Converted 14_segmentation_postprocess.ipynb.
Converted 15_segmentation_tta.ipynb.
Converted 15_segmentation_utils.ipynb.
Converted segmentation_training.ipynb.


### fin